In [ ]:
# run in Chameleon Jupyter environment
from chi import context, server
import chi
import os

context.version = "1.0" 
context.choose_project()  # Select the correct project
context.choose_site(default="KVM@TACC")
username = os.getenv('USER') # exp resources will have this suffix

In [ ]:
# run in Chameleon Jupyter environment
s = server.Server(
    f"node-persist-project17", 
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.large"
)
s.submit(idempotent=True)

In [ ]:
# run in Chameleon Jupyter environment
s.associate_floating_ip()

In [ ]:
# run in Chameleon Jupyter environment
s.refresh()
s.show(type="widget")

In [ ]:
# run in Chameleon Jupyter environment
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by MLFlow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"},
  {'name': "allow-8501", 'port': 8501, 'description': "Enable TCP port 8501 (used by StreamLiT Web UI)"}
]

os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:
  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

In [ ]:
# run in Chameleon Jupyter environment
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

In [ ]:
s.execute("git clone https://github.com/phoenix1881/Medical-Chatbot-MLOps.git")

In [ ]:
s.execute("openstack object store container create object-persist-project17")

In [ ]:
s.execute("openstack volume create --size 50 block-persist-project17")
s.execute("openstack server add volume node-persist-project17 block-persist-project17")

In [ ]:
s.execute("sudo mkfs.ext4 /dev/vdb && sudo mkdir -p /mnt/block && sudo mount /dev/vdb /mnt/block")

In [ ]:
s.execute("cd geetha/scripts/; bash block_mount.sh")

In [ ]:
s.execute("cd geetha/scripts/; bash object_mount.sh")

In [ ]:
s.execute("cd geetha/scripts/; bash run_etl.sh")

In [ ]:
s.execute("cd geetha/scripts/; bash run_retraining_etl.sh")

In [ ]:
s.execute("HOST_IP=$(curl --silent http://169.254.169.254/latest/meta-data/public-ipv4 ) docker compose -f ~/Medical-Chatbot-MLOps/geetha/docker/docker-compose-persistant-storage.yaml up -d;")

In [ ]:
s.execute("cd geetha/docker/;  docker compose -f docker-compose-dashboard.yaml up --build -d;")